# Gallery Example: Finite Repairmen Model

This is a classic finite population model representing a repairmen system:
- **Machines**: Finite number of machines that can break down
- **Repairmen**: Limited number of servers to fix machines
- **Topology**: Machines cycle between working (delay) and being repaired (queue)
- **Population**: Finite and fixed number of jobs (machines)

This model is fundamental for:
- Manufacturing systems
- Computer systems reliability
- Any finite population service system

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.fluid.*
import jline.solvers.mva.*
GlobalConstants.setVerbose(VerboseLevel.STD)

In [ ]:
fun gallery_repairmen(nservers=1, seed=23000): Network {    """Create finite repairmen model        Parameters:    - nservers: Number of repair servers (default 1)    - seed: Random seed for reproducibility (default 23000)    """    model = Network("Finite repairmen CQN")        # Set random seed    random.seed(seed)    np.random.seed(seed)        # Block 1: nodes    repair_queue = Queue(model, "Queue1", SchedStrategy.PS)    repair_queue.setNumberOfServers(nservers)        working_delay = Delay(model, "Delay1")  # Machines working (operational time)        # Block 2: classes    # Random population of machines (between 3 and 13)    population = int(random.random() * 10 + 3)    jobclass = ClosedClass(model, "Class1", population, repair_queue, 0)        # Set service times    repair_rate = random.random() + 1  # Random repair rate between 1 and 2    repair_queue.setService(jobclass, Exp.fitMean(repair_rate))        working_delay.setService(jobclass, Exp.fitMean(2.0))  # Mean time between failures        # Block 3: topology - simple cycle    P = model.initRoutingMatrix()    P.addRoute(jobclass, repair_queue, working_delay, 1.0)  # Repaired -> Working    P.addRoute(jobclass, working_delay, repair_queue, 1.0)  # Broken -> Repair        model.link(P)        return model, population, nservers# Create the modelmodel, population, nservers = gallery_repairmen(nservers=2, seed=23000)println(f"Machines: {population}")println(f"Repairmen: {nservers}")

## Model Interpretation

- **Queue1**: Repair facility where broken machines wait to be fixed
- **Delay1**: Operational state where machines work until they break down
- **Population**: Fixed number of machines in the system
- **Servers**: Number of repair technicians available

Key metrics:
- **Queue length at repair**: Average number of machines waiting/being repaired
- **Queue length at delay**: Average number of operational machines  
- **Utilization**: Fraction of time repair facility is busy
- **Throughput**: Rate of completions (repairs per time unit)

In [ ]:
// Solve with multiple solvers
println("\n=== Solver Results ===")
// MVA Solver (exact for closed models)
val solver_mva = MVA(model)
val avg_table_mva = solver_mva.avgTable
println("\nMVA Solver:")
println(avg_table_mva)
// CTMC Solver 
val solver_ctmc = CTMC(model)
val avg_table_ctmc = solver_ctmc.avgTable
println("\nCTMC Solver:")
println(avg_table_ctmc)
// NC Solver
val solver_nc = NC(model)
val avg_table_nc = solver_nc.avgTable
println("\nNC Solver:")
println(avg_table_nc)